# Install packages

In [24]:
!pip install scenic
!pip install einops
# !pip install -qq medmnist
!pip install --upgrade keras
!pip install keras-tuner
!pip install gdown

In [10]:
# Reset the plotly graph
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Initialize plotly notebook mode
init_notebook_mode(connected=True)

In [11]:
import cv2 as tf
print(tf.__version__)


4.10.0


In [7]:
import tensorflow as tf
import numpy as np
# import matplotlib.pyplot as plt
# import skvideo.io
import pandas as pd
import cv2
import os

# Reload data from pkl

#### Video

In [11]:
import pickle
with open('/kaggle/input/3july-2-pkll/3July_2/ab_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_videos.append(array_a_loaded)
  except:
    print('end!')
    break

end!


In [12]:
with open('/kaggle/input/3july-2-pkll/3July_2/nor_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_videos.append(array_a_loaded)
  except:
    print('end!')
    break

end!


#### Dims

In [13]:
with open('/kaggle/input/3july-2-pkll/3July_2/ab_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break
with open('/kaggle/input/3july-2-pkll/3July_2/nor_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break

end!
end!


# Modelling pre-process <a class="anchor"  id="modelpre"></a>

In [14]:
human_action_dataset = np.asarray( ab_videos + nor_videos )

In [15]:
# ab = 1
# nor = 0
labels = np.concatenate([np.ones(len(ab_videos)),np.zeros(len(nor_videos))])

In [16]:
ab_videos_dims_df = pd.DataFrame(ab_video_dims,columns=['frames','width','height','channel'])
nor_videos_dims_df = pd.DataFrame(nor_video_dims,columns=['frames','width','height','channel'])

In [17]:
from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(human_action_dataset,labels,test_size=0.25,shuffle=True,random_state=42)
# X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,test_size=0.8,shuffle=True,random_state=42)


X_train,X_test,y_train,y_test = train_test_split(human_action_dataset,labels,test_size=0.2,shuffle=True,random_state=42,stratify=labels)
X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,shuffle=True,random_state=42)


In [15]:
import os
import io
import imageio
import ipywidgets
import numpy as np
import tensorflow as tf  # for data preprocessing only
import keras
from keras import layers, ops, regularizers

# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

# Trans body

In [16]:
# Setting seed for reproducibility
SEED = 77
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
tf.random.set_seed(SEED)

# DATA
BATCH_SIZE = 4
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (40, 224, 224, 3)
NUM_CLASSES = 2

# OPTIMIZER
# LEARNING_RATE = 1e-4 #Default
LEARNING_RATE = 1e-4 #NOTE: CHANGE THIS
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 40

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 64
NUM_HEADS = 2
NUM_LAYERS = 2

In [20]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """
      Preprocess the frames tensors and parse the labels.
      This code convert the framess into float for normalization hence
      we should add a convert dtype code back to the model
    """
    # Preprocess images
    frames = tf.image.convert_image_dtype(frames[..., tf.newaxis],  tf.float32,)
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

# Training set
trainloader = prepare_dataloader(X_train, y_train, "train")
testloader = prepare_dataloader(X_test, y_test, "test")
validloader = prepare_dataloader(X_valid, y_valid, "test")

# Further testing set
# testloader2 = prepare_dataloader(human_action_dataset, labels, "test2")

In [22]:
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable(package="Custom", name="TubeletEmbedding")
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim, # embed_dim = 64
            kernel_size=patch_size, # patch_size = (8, 8, 8)
            strides=patch_size, 
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

@register_keras_serializable(package="Custom", name="PositionalEncoder")
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape # input_shape =(numVid,3920,64)
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1) # Set positions

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions # Concat the position with it token
        return encoded_tokens

In [18]:
# Base line
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,# (40, 224, 224, 3)
    transformer_layers=NUM_LAYERS,# num_layers = 2
    num_heads=NUM_HEADS, # num_heads = 2
    embed_dim=PROJECTION_DIM,# embed_dim = 64
    layer_norm_eps=LAYER_NORM_EPS, # le-6
    num_classes=NUM_CLASSES,# 2(normal, abnormal)
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Multi-head
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1 # NOTE: dropout
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=ops.gelu),
                layers.Dense(units=embed_dim, activation=ops.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Final ViViT Model
NOTE: 
1. This was run after the hyperparameter completed
2. Findings: Drop out rate = 0.4 & learning rate = 1e-4 had score the highes val_accuracy

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def run_experiment():
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

    # Train the model
    history = model.fit(
        trainloader,
        epochs=EPOCHS,
        validation_data=validloader,
        callbacks=[early_stopping],
    )

    # Evaluate on test set
    print("--------------------------------------------------")
    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history, model

history, model = run_experiment()


Number of devices: 2
Epoch 1/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 50s 150ms/step - accuracy: 0.5694 - loss: 0.8367 - top-5-accuracy: 1.0000 - val_accuracy: 0.5500 - val_loss: 0.6385 - val_top-5-accuracy: 1.0000
Epoch 2/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 141ms/step - accuracy: 0.5690 - loss: 0.6775 - top-5-accuracy: 1.0000 - val_accuracy: 0.6250 - val_loss: 0.6184 - val_top-5-accuracy: 1.0000
Epoch 3/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 140ms/step - accuracy: 0.5845 - loss: 0.6700 - top-5-accuracy: 1.0000 - val_accuracy: 0.6500 - val_loss: 0.5927 - val_top-5-accuracy: 1.0000
Epoch 4/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 141ms/step - accuracy: 0.6176 - loss: 0.6354 - top-5-accuracy: 1.0000 - val_accuracy: 0.6750 - val_loss: 0.5583 - val_top-5-accuracy: 1.0000
Epoch 5/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 139ms/step - accuracy: 0.7037 - loss: 0.5623 - top-5-accuracy: 1.0000 - val_accuracy: 0.9250 - val_loss: 0.3686 - val_top-5-accuracy: 1.0000
Epoch 6/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 142ms/step

## Loss Graph

In [41]:
import plotly.graph_objs as go
import plotly.subplots as sp

# Sample data (replace these with your actual loss values)
loss2 = history.history['loss']
val_loss = history.history['val_loss']

epochs = list(range(1, len(loss2) + 1))

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))

# Remove accuracy subplot by creating only one subplot for loss
fig = sp.make_subplots(rows=1, cols=1, subplot_titles=('Loss',))

# Add traces for loss
fig.add_trace(
    go.Scatter(x=epochs, y=loss2, mode='lines', name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=val_loss, mode='lines', name='Test Loss', line=dict(color='red')),
    row=1, col=1
)

# Update layout
fig.update_layout(
    title_text='ViViT Training and Validation Loss',
    showlegend=True,
    xaxis_title='Epoch',
    yaxis_title='Value'
)

# Update xaxis and yaxis titles for the loss subplot
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_yaxes(title_text='Loss', dtick=0.1, row=1, col=1)  # Change dtick to desired step size for Loss

# Show figure
fig.show()

## Accuracy Graph

In [33]:
 _, accuracy, _ = model.evaluate(testloader)

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9288 - loss: 0.2170 - top-5-accuracy: 1.0000


In [39]:
from sklearn.metrics import classification_report
def calculate_metrics(model, testloader):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frames, labels in testloader:
        preds = model.predict(frames)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    # Calculate metrics
    report = classification_report(y_true, y_pred, target_names=label_dict.values(), output_dict=True)
    
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    return precision, recall, f1_score


precision, recall, f1_score = calculate_metrics(model,testloader)
print(f"Accuracy: {round(accuracy * 100, 2)}%")
print(f'Precision: {round(precision * 100,2)}%')
print(f'Recall: {round(recall * 100,2)}%')
print(f'f1_score: {round(f1_score * 100,2)}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
Accuracy: 95.0%
Precision: 91.52%
Recall: 91.25%
f1_score: 91.25%


## Confusion Matrix

In [ ]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader], axis=0)

# Get the predictions
predictions = model.predict(testloader)
# predictions = model.predict(validloader)
predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step


# Model Before Tuned
- Not tuned model

NOTE: Accuracy score was good however the fluctuation of the vlidation loss is terrible

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def run_experiment():
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

    # Train the model
    history = model.fit(
        trainloader,
        epochs=EPOCHS,
        validation_data=validloader,
        callbacks=[early_stopping],
    )

    # Evaluate on test set
    print("--------------------------------------------------")
    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%") #NOTE: If the true label under the top 5 then consider correct prediction

    return history, model

history, model = run_experiment()


Number of devices: 2
Epoch 1/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 47s 153ms/step - accuracy: 0.5635 - loss: 0.7997 - top-5-accuracy: 1.0000 - val_accuracy: 0.6250 - val_loss: 0.6479 - val_top-5-accuracy: 1.0000
Epoch 2/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 143ms/step - accuracy: 0.5794 - loss: 0.7109 - top-5-accuracy: 1.0000 - val_accuracy: 0.6750 - val_loss: 0.5937 - val_top-5-accuracy: 1.0000
Epoch 3/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 142ms/step - accuracy: 0.5771 - loss: 0.6728 - top-5-accuracy: 1.0000 - val_accuracy: 0.6500 - val_loss: 0.5635 - val_top-5-accuracy: 1.0000
Epoch 4/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 142ms/step - accuracy: 0.6727 - loss: 0.6000 - top-5-accuracy: 1.0000 - val_accuracy: 0.8500 - val_loss: 0.4365 - val_top-5-accuracy: 1.0000
Epoch 5/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 142ms/step - accuracy: 0.7804 - loss: 0.4851 - top-5-accuracy: 1.0000 - val_accuracy: 0.9000 - val_loss: 0.2984 - val_top-5-accuracy: 1.0000
Epoch 6/40
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 142ms/step

## Loss Graph

In [31]:
import plotly.graph_objs as go
import plotly.subplots as sp

# Sample data (replace these with your actual loss values)
loss2 = history.history['loss']
val_loss = history.history['val_loss']

epochs = list(range(1, len(loss2) + 1))


# Remove accuracy subplot by creating only one subplot for loss
fig = sp.make_subplots(rows=1, cols=1, subplot_titles=('Loss',))

# Add traces for loss
fig.add_trace(
    go.Scatter(x=epochs, y=loss2, mode='lines', name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=val_loss, mode='lines', name='Test Loss', line=dict(color='red')),
    row=1, col=1
)

# Update layout
fig.update_layout(
    title_text='ViViT Training and Validation Loss',
    showlegend=True,
    xaxis_title='Epoch',
    yaxis_title='Value'
)

# Update xaxis and yaxis titles for the loss subplot
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_yaxes(title_text='Loss', dtick=0.1, row=1, col=1)  # Change dtick to desired step size for Loss

# Show figure
fig.show()

## Accuracy Graph

In [37]:
# Average trian and validation accuracy

acc2 = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Calculate the average accuracy
avg_train_acc = np.mean(acc2)
avg_val_acc = np.mean(val_acc)

print(f"Average Train Accuracy: {avg_train_acc*100}%")
print(f"Average Validation Accuracy: {avg_val_acc*100}%")


Average Train Accuracy: 89.5192311360286%
Average Validation Accuracy: 90.48076913906978%


# Hyperparameter Tuning

In [23]:
# Base line
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    dropout_rate,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout_rate
        )(x1, x1)


        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=ops.gelu),
                layers.Dense(units=embed_dim, activation=ops.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [24]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def build_model(hp):
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    DROPOUT_RATE = hp.Float('dropout_rate', min_value=0.2, max_value=0.8, step=0.1) #NOTE: Total 7
    
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
            dropout_rate = DROPOUT_RATE,
        )
        
        LEARNING_RATE = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5]) #NOTE: total 4
        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    return model
#  model = run_experiment()


In [ ]:
tuner = kt.GridSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=50,
    executions_per_trial=1,
    overwrite=True,
    directory='my_dir',
    project_name="ViViT_GridSearch",
)

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
dropout_rate (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.8, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}


In [8]:
tuner.search(trainloader, epochs=40, validation_data=validloader)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter grid serachis completed.The optimal parameter for
dropput is {round(best_hps.get('dropout_rate'),1)} and for the learnign rate is {best_hps.get('learning_rate')}""")

Trial 28 Complete [00h 09m 51s]
val_accuracy: 0.9051000223920929

Best val_accuracy So Far: 0.949999988079071
Total elasped time: 01h 42m 45s

The hyperparameter grid serachis completed.The optimal parameter for 
dropput is 0.4 and for the learnign rate is 1e-4


# Save model

In [23]:
model.save('ViViT_24Jul.keras')

# Demostration

In [ ]:
model = tf.keras.models.load_model('../Saved_model/ViViT_3July_2.keras')

In [ ]:
"""
Preprocess before modeling 
1. Trim frames (max 40 frames)
2. Resizing (224,224)
"""
def load_all_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return {'frames': None, 'frames_dim': None, 'success': False}

    frames_dims = []
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        h, w, c = frame.shape
        frames_dims.append([0, h, w, c])
        frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_CUBIC)
        frames.append(frame)

    cap.release()
    return {'frames': np.asarray(frames), 'frames_dim': frames_dims, 'success': True}

def trim_video_frames(video, max_frame):
    '''
    Args:
        video: video (collection of frames)
        max_frame: max number of frames
    '''
    f, _, _, _ = video.shape
    startf = f // 2 - max_frame // 2
    return video[startf:startf + max_frame, :, :, :]

def preprocess_single_video(video):
    video = trim_video_frames(video, 40)
    video = tf.image.convert_image_dtype(video, tf.float32)
    return video

In [ ]:

label_dict = {0: 'Normal', 1: 'Abnormal'}
video_path = '../Dataset/Test_dataset/abnormal/video_254_flip.avi'

load_data = load_all_frames(video_path)
if not load_data['success']:
    print('Video is corrupt!!')
else:
    video = load_data['frames']
    preprocessed_video = preprocess_single_video(video)
    output = model.predict(tf.expand_dims(preprocessed_video, axis=0))[0]
    print(output)
    pred = np.argmax(output, axis=0)
    print(label_dict[pred])